In [1]:
from pycocotools.coco import COCO
from mmengine import dump, load
from pprint import pprint
from collections import defaultdict
from copy import deepcopy

from utils import (update_img_info, exclude_imgs, 
                   merge_coco, extract_img_info, gen_new_coco, 
                   train_exclude_img_ids, test_exclude_img_ids)

## 排除部分纠正不了的脏数据

In [52]:
raw_ins_train2 = COCO('../pasta/raw/instances_train.json')
raw_rel_train2 = COCO('../pasta/raw/relations_train.json')
pasta_rel_train2 = COCO('../pasta/raw/pastas_train.json')

raw_ins_train2_merged = COCO('../pasta/merge/instances_train2_merged.json')
raw_rel_train2_merged = COCO('../pasta/merge/relations_train2_merged.json')
pasta_rel_train2_merged = COCO('../pasta/merge/pastas_train2_merged.json')

raw_ins_train = COCO('../../annotations/instances_train.json')
raw_rel_train = COCO('../../annotations/relations_train.json')
raw_ins_test = COCO('../../annotations/instances_test.json')
raw_rel_test = COCO('../../annotations/relations_test.json')

raw_ins_train_merged = COCO('../../annotations/instances_train_merged.json')
raw_rel_train_merged = COCO('../../annotations/relations_train_merged.json')
raw_ins_test_merged = COCO('../../annotations/instances_test_merged.json')
raw_rel_test_merged = COCO('../../annotations/relations_test_merged.json')

loading annotations into memory...
Done (t=0.58s)
creating index...
index created!
loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
loading annotations into memory...
Done (t=0.50s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [53]:
print('raw_ins_train')
new_ins_train, exclude_success_ins_train = exclude_imgs(raw_ins_train, exclude_img_ids=train_exclude_img_ids)
print('raw_rel_train')
new_rel_train, exclude_success_rel_train = exclude_imgs(raw_rel_train, exclude_img_ids=train_exclude_img_ids)
print('raw_ins_test')
new_ins_test, exclude_success_ins_test = exclude_imgs(raw_ins_test, exclude_img_ids=test_exclude_img_ids)
print('raw_rel_test')
new_rel_test, exclude_success_rel_test = exclude_imgs(raw_rel_test, exclude_img_ids=test_exclude_img_ids)

print('raw_ins_train_merged')
new_ins_train_merged, exclude_success_ins_train_merged = exclude_imgs(raw_ins_train_merged, exclude_img_ids=train_exclude_img_ids)
print('raw_rel_train_merged')
new_rel_train_merged, exclude_success_rel_train_merged = exclude_imgs(raw_rel_train_merged, exclude_img_ids=train_exclude_img_ids)
print('raw_ins_test_merged')
new_ins_test_merged, exclude_success_ins_test_merged = exclude_imgs(raw_ins_test_merged, exclude_img_ids=test_exclude_img_ids)
print('raw_rel_test_merged')
new_rel_test_merged, exclude_success_rel_test_merged = exclude_imgs(raw_rel_test_merged, exclude_img_ids=test_exclude_img_ids)

successfully exclude 0/152 images
creating index...
index created!
successfully exclude 0/152 images
creating index...
index created!
successfully exclude 0/15 images
creating index...
index created!
successfully exclude 0/15 images
creating index...
index created!


In [51]:
dump(new_ins_train.dataset, 'instances_train.json')
dump(new_rel_train.dataset, 'relations_train.json')
dump(new_ins_test.dataset, 'instances_test.json')
dump(new_rel_test.dataset, 'relations_test.json')

dump(new_ins_train_merged.dataset, 'instances_train_merged.json')
dump(new_rel_train_merged.dataset, 'relations_train_merged.json')
dump(new_ins_test_merged.dataset, 'instances_test_merged.json')
dump(new_rel_test_merged.dataset, 'relations_test_merged.json')

### 同步更新image_info

In [54]:
train_img_info = extract_img_info(raw_rel_train)
test_img_info = extract_img_info(raw_rel_test)

creating index...
index created!
creating index...
index created!


In [55]:
dump(train_img_info.dataset, 'image_info_train.json')
dump(test_img_info.dataset, 'image_info_test.json')

## 合并更新标注

In [5]:
coco1 = COCO('')
coco2 = COCO('')

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [6]:
merged = merge_coco(coco1, coco2, rebuild_ann_id=False)

creating index...
index created!
creating index...
index created!
creating index...
index created!


In [7]:
dump(merged.dataset, '')

## 更新被重标注的文件列表

#### train

In [5]:
# manualed_train = COCO('../../annotations/mini/instances_train.json')
# manualed_train = COCO('../box_merge/iou_70/instances_train_manual_1.json')
# manualed_train = COCO('../box_merge/iou_70/instances_train_manual_2.json')
manualed_train = COCO('../subset/rare/instances_train_merged.json')

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [6]:
# train
raw = set(load('manualed_train.bak.json'))
new = sorted(list(raw | set(manualed_train.imgs.keys())))
dump(new, 'manualed_train.json')
print(f'{len(raw)} -> {len(new)}')
# 再手动改一下文件名

3051 -> 3061


#### test

In [44]:
manualed_test = COCO('../box_merge/iou_70/instances_test_manual_1.json')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [45]:
# test
raw = set(load('manualed_test.bak.json'))
new = sorted(list(raw | set(manualed_test.imgs.keys())))
dump(new, 'manualed_test.json')
print(f'{len(raw)} -> {len(new)}')
# 再手动改一下文件名

0 -> 26


## 剔除重复rel

In [7]:
# train_rel = COCO('../../annotations/relations_train_merged.json')
# test_rel = COCO('../../annotations/relations_test_merged.json')

# train_rel = COCO('../../annotations/mini/relations_train.json')
# test_rel = COCO('../../annotations/mini/relations_test.json')

test_rel = COCO('../box_merge/iou_70/merged_rel_test.json')

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


In [8]:
def remove_duplicate_rel(coco: COCO):
    total_anns = set()
    new_ann_ids = []
    for ann in coco.dataset['annotations']:
        key = (ann['image_id'], ann['subject_id'], ann['object_id'], ann['category_id'])
        if key not in total_anns:
            total_anns.add(key)
            new_ann_ids.append(ann['id'])
    num_raw, num_new = len(coco.dataset['annotations']), len(new_ann_ids)
    duplicate = num_raw - num_new
    if duplicate == 0:
        print('没有重复标注')
        return coco
    else:
        print(f'剔除了{duplicate}个重复标注，{num_raw} -> {num_new}')
        return gen_new_coco(coco, new_anns=coco.loadAnns(new_ann_ids))

In [9]:
# train_rel = remove_duplicate_rel(train_rel)
test_rel = remove_duplicate_rel(test_rel)

没有重复标注


In [12]:
sum(ann['category_id'] != 58 for ann in test_rel.anns.values())

28831

In [10]:
len(test_rel.anns)

33041

In [11]:
# dump(train_rel.dataset, 'relations_train.json')
dump(test_rel.dataset, 'relations_test.json')

## 处理太小的bbox

In [2]:
# train_ins = COCO('../../annotations/instances_train.json')
# test_ins  = COCO('../../annotations/instances_test.json')

# train_ins = COCO('../../annotations/raw/instances_train.json')
# test_ins  = COCO('../../annotations/raw/instances_test.json')

# train_ins = COCO('../../annotations/instances_train_merged.json')
# test_ins  = COCO('../../annotations/instances_test_merged.json')

train_ins = COCO('../box_merge/iou_70/merged_ins_train.json')
test_ins  = COCO('../box_merge/iou_70/merged_ins_test.json')

loading annotations into memory...
Done (t=0.50s)
creating index...
index created!
loading annotations into memory...
Done (t=0.20s)
creating index...
index created!


In [3]:
def bbox_min_hw(coco: COCO, mode: str = 'none', min_hw: int = 5):
    invalid = set()
    new_anns = []
    for id, ann in coco.anns.items():
        ann = deepcopy(ann)
        x, y, w, h = ann['bbox']
        if w < min_hw or h < min_hw:
            invalid.add(id)
            if mode == 'keep':
                img_info = coco.loadImgs(ann['image_id'])[0]
                img_h, img_w = img_info['height'], img_info['width']

                need_w = max(0, min_hw - w)
                need_h = max(0, min_hw - h)

                new_w = w + need_w
                new_h = h + need_h
                new_x = max(0, x - need_w // 2)
                new_y = max(0, y - need_h // 2)
                new_x -= max(0, new_x + new_w - img_w)
                new_y -= max(0, new_y + new_h - img_h)

                ann['bbox'] = [new_x, new_y, new_w, new_h]
            if mode == 'delete':
                continue
        new_anns.append(ann)

    if mode == 'none' or len(invalid) == 0:
        print('return raw coco ann')
        new_coco = coco
    else:
        new_coco = gen_new_coco(coco, new_anns=new_anns)
    return invalid, new_coco

In [4]:
invalid_train, new_train_ins = bbox_min_hw(train_ins, mode='keep')
len(invalid_train)

creating index...
index created!


69

In [5]:
invalid_test, new_test_ins = bbox_min_hw(test_ins, mode='keep')
len(invalid_test)

creating index...
index created!


12

In [6]:
# dump(new_train_ins.dataset, 'instances_train.json')
dump(new_test_ins.dataset, 'merged_ins_test.json')